# Projeto 2 - Ciência dos Dados

Nome: Carlos Eduardo Dip

Nome: Gianluca Lazzaris Giudici

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import numpy as np
import os.path
import pandas as pd
import json
import re
import string
import functools
import operator
from random import shuffle

try:
    import emoji
except:
    !pip install emoji --upgrade
    import emoji

___
### Coleta de Dados:

Os dados foram coletados em outro notebook, chamado Coletor.ipynb. Abaixo, será importado o excel com os dados já classificados.

___

## Limpeza, aquisição e organização dos dados

In [2]:
data_training = pd.read_excel('data/Uber.xlsx')
data_validation = pd.read_excel('data/Uber.xlsx', 'Teste')

In [3]:
data_training_R = data_training.loc[data_training.Classificacao == 1]
data_training_NR = data_training.loc[data_training.Classificacao == 0]

TrainingString_Relevant = ''
TrainingString_NotRelevant = ''

In [4]:
def check_laughter(word):
    for letter in word:
        if letter != 'k':
            return False
    return True

def cleanup(text):

    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text_split = ' '.join(word for word in text_split)
    
    text = ' '.join(word for word in text_split.split() if not word.startswith('https'))
    
    
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = text_subbed.lower()
    text_out = ' '
    
    
    for word in text_subbed.split():
        if check_laughter(word):
            text_out += ' haha'
        else:
            text_out += ' ' + word
    
    
    return text_out    

In [5]:
### --- Cleanup function demo --- ###
demo_string_ =  """
                X gon give it to ya
                Fuck wait for you to get it on your own
                X gon deliver to ya
                Knock knock, open up the door, it's real
                Wit the non-stop, pop pop and stainless steel
                Go hard gettin busy wit it
                https://img.imgur/1337, uber blah blah kkk, 👨🏿😷😷
                https://img.imgur/1337, uber blah blah kkk, 👨🏿😷😷
                """

print(cleanup(demo_string_))
print(cleanup(demo_string_).split())

  x gon give it to ya fuck wait for you to get it on your own x gon deliver to ya knock knock open up the door it s real wit the non stop pop pop and stainless steel go hard gettin busy wit it uber blah blah haha 👨🏿 😷 😷 uber blah blah haha 👨🏿 😷 😷
['x', 'gon', 'give', 'it', 'to', 'ya', 'fuck', 'wait', 'for', 'you', 'to', 'get', 'it', 'on', 'your', 'own', 'x', 'gon', 'deliver', 'to', 'ya', 'knock', 'knock', 'open', 'up', 'the', 'door', 'it', 's', 'real', 'wit', 'the', 'non', 'stop', 'pop', 'pop', 'and', 'stainless', 'steel', 'go', 'hard', 'gettin', 'busy', 'wit', 'it', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷']


In [6]:
for phrase in data_training_R.Treinamento:
    TrainingString_Relevant += phrase
for phrase in data_training_NR.Treinamento:
    TrainingString_NotRelevant += phrase    
    
cleanup(TrainingString_NotRelevant)
cleanup(TrainingString_Relevant)

TrainingSeries_Relevant = pd.Series(TrainingString_Relevant.split()).value_counts(True)
TrainingSeries_NotRelevant = pd.Series(TrainingString_NotRelevant.split()).value_counts(True)

for word in ['o', 'de', 'e', 'que', 'do', 'a', 'pra', 'um']:
    del TrainingSeries_NotRelevant[word]
    del TrainingSeries_Relevant[word]

LaplaceConstant = 0.0000001

### --- Laplace Smoothing --- ###
TrainingSeries_Relevant += LaplaceConstant
TrainingSeries_NotRelevant += LaplaceConstant

### --- Log of everything to prevent underflow --- ###
TrainingSeries_Relevant = np.log(TrainingSeries_Relevant)
TrainingSeries_NotRelevant = np.log(TrainingSeries_NotRelevant)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
P_Relevant = len(TrainingSeries_Relevant)/(len(TrainingSeries_Relevant)+len(TrainingSeries_NotRelevant))
P_NotRelevant = len(TrainingSeries_NotRelevant)/(len(TrainingSeries_Relevant)+len(TrainingSeries_NotRelevant))
print(P_NotRelevant, P_Relevant)

0.47374301675977654 0.5262569832402234


In [8]:
def evaluate_relevance(tweet = ""):
    
    ## -- Cleaning tweet
    text = cleanup(tweet)
    
#     ## -- Evaluates likelihood
    sumR = 0
    sumNR = 0
    
    for word in text.split():
    
        if word in TrainingSeries_Relevant: 
            sumR += TrainingSeries_Relevant[word]
        else:
            sumR += np.log(LaplaceConstant)
        if word in TrainingSeries_NotRelevant:
            sumNR += TrainingSeries_NotRelevant[word]
        else:
            sumNR += np.log(LaplaceConstant)
    
    sumR += np.log(P_Relevant)
    sumNR += np.log(P_NotRelevant)
    
    return sumR>sumNR


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [9]:
evaluations = [int(evaluate_relevance(tweet)) for tweet in data_validation.Teste]

In [10]:
data_validation['bayes'] = evaluations

In [11]:
Accuracy = {'True-True':0,
            'True-False':0,
            'False-True':0,
            'False-False':0
           }


for tweet,evaluation in zip(data_validation.classificacao, data_validation.bayes):
    if tweet and evaluation:
        Accuracy['True-True'] += 1
    elif tweet and not evaluation:
        Accuracy['True-False'] += 1
    elif not tweet and evaluation:
        Accuracy['False-True'] += 1
    elif not tweet and not evaluation:
        Accuracy['False-False'] += 1

Accuracy_Normalized = {}
S = sum(Accuracy.values())
for k,v in Accuracy.items():
    Accuracy_Normalized[k] = v/S
Accuracy_Normalized['Final-Accuracy'] = str(round((Accuracy_Normalized['True-True']+Accuracy_Normalized['False-False'])*100, 3)) + '%'
Accuracy_Normalized

{'True-True': 0.2914572864321608,
 'True-False': 0.23115577889447236,
 'False-True': 0.24623115577889448,
 'False-False': 0.23115577889447236,
 'Final-Accuracy': '52.261%'}

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) **Explicação da técnica para lidar com palavras novas**